Para empezar con el proyecto cargo las librerias a usar

In [2]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

import Funciones_aux as fa
from ast import literal_eval 


Ahora cargo los datos de los .csv

In [13]:
ruta_movies = '../data/movies_dataset.csv'
movies = pd.read_csv(ruta_movies)

ruta_credits = '../data/credits.csv'
credits = pd.read_csv(ruta_credits)

C:\Users\biott\AppData\Local\Temp\ipykernel_11632\3807620164.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(ruta_movies)


Combino los dos archivos .csv en un solo archivo para asi poder trabajar con ambos de una forma mas sencilla

In [14]:
movies_full = pd.concat([movies, credits], axis=1, ignore_index=False)

### Transformaciones

Ahora empiezo a limpiar y trabajar el dataframe

In [15]:
#Primero relleno los NA de revenue y budget con 0
movies_full['revenue'].fillna(0, inplace=True)
movies_full['budget'] = pd.to_numeric(movies_full['budget'], errors='coerce').fillna(0) #Chequeo que sean numeros ya que estaba tomando algunos valores que no lo eran y no permitia funcionar al codigo

#Empiezo a trabajar con la columna release_date
#Sigo por borrar las columnas que no tenga una fecha valida
movies_full = movies_full.dropna(subset=['release_date'])
#Me aseguro de que todas las fechas de esta esten en el formato correcto
movies_full['release_date'] = pd.to_datetime(movies_full['release_date'], errors='coerce')
movies_full = movies_full.dropna(subset=['release_date'])
#Creo una columna day_of_week y una columna month para usar luego en las funciones
movies_full['month'] = movies_full['release_date'].dt.month.astype(int)
movies_full['day_of_week'] = movies_full['release_date'].dt.dayofweek.astype(int)

#Y genero la columna release_year con los valores de año de release_date
movies_full['release_year'] = movies_full['release_date'].dt.year

#Despues elimino las columnas que no voy a utilizar
movies_full.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

#Ahora creo la columna return con los valores de revenue/budget o 0 si no es posible
movies_full['return'] = movies_full.apply(lambda fila: fila['revenue']/ fila['budget'] if fila['budget'] > 0 else 0, axis=1)

#Y ahora desanido las columnas que contienen diccionarios y listas usando las funciones auxiliares que cree
movies_full['belongs_to_collection'] = movies_full['belongs_to_collection'].apply(fa.get_collection_name)
movies_full['genres'] = movies_full['genres'].apply(fa.get_genre_names)
movies_full['production_companies'] = movies_full['production_companies'].apply(fa.get_companies_names)
movies_full['production_countries'] = movies_full['production_countries'].apply(fa.get_country_iso)
movies_full['spoken_languages'] = movies_full['spoken_languages'].apply(fa.get_language_iso)



C:\Users\biott\AppData\Local\Temp\ipykernel_11632\3876144179.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_full['revenue'].fillna(0, inplace=True)


Por ultimo guardo una muestra aleatoria del dataset original

In [18]:
movies_sample = movies_full.sample(frac=0.2, random_state=42)
ruta_movies_sample = '../data/movies_sample.csv'
movies_sample.to_csv(ruta_movies_sample, index=False)